In [1]:
ls

drive/  sample_data/


## read data

In [2]:
import pandas as pd


df = pd.read_csv('/content/drive/MyDrive/kaggle/Hotel_Reviews.csv')

len(df)


515738

In [3]:

# df.columns

df.all_review = df.apply(lambda x:f'{x["Positive_Review"]} , {x["Negative_Review"]}  ' ,axis=1)

sentences = [r for r in df.all_review]

len(sentences)

<ipython-input-3-3a434047499c>:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.all_review = df.apply(lambda x:f'{x["Positive_Review"]} , {x["Negative_Review"]}  ' ,axis=1)


515738

In [4]:
for r in sentences[:10]:
  print()
  print()
  print(r)
  print()
  print()



 Only the park outside of the hotel was beautiful  ,  I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day SO i had to check out the next day before 11 o clock in order 

In [9]:
!pip install faiss-gpu #cpu

In [10]:
import faiss

In [11]:
!pip install -U sentence-transformers

In [5]:
import torch
torch.cuda.is_available()


from sentence_transformers import SentenceTransformer , util
import numpy as np
# initialize sentence transformer model
#model = SentenceTransformer('bert-base-nli-mean-tokens') #encoding model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
device = torch.device('cuda' ) # Define the device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Define the device
model.to(device)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [13]:
filtered_sentences = list({x for x in sentences[:]})
len(filtered_sentences)

499288

In [14]:
%%time
sentence_embeddings = model.encode(filtered_sentences,convert_to_tensor=True)
sentence_embeddings.shape

CPU times: user 10min 18s, sys: 26.8 s, total: 10min 44s
Wall time: 5min 43s


torch.Size([499288, 768])

In [15]:
len(sentence_embeddings)

499288

## semantic_search

In [16]:
def semantic_search(query, sentence_embeddings, top_k=3 ):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)
    query_embedding = query_embedding.to(device) # Move query embedding to the same device as the model

    # Compute cosine similarities between the query and the documents
    cosine_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings.to(device))[0] # Move sentence embeddings to the same device

    # Move cosine_scores to CPU before applying NumPy functions
    cosine_scores = cosine_scores.cpu()

    # Find the top_k highest scores
    top_results = np.argpartition(-cosine_scores, range(top_k))[:top_k]

    print(f"Query: {query}\nTop {top_k} most similar documents:")
    for idx in top_results:
        print("*"*20)
        print(f"Score: {cosine_scores[idx]:.4f}")
        print()
        print(f"{sentences[idx]}")
        print()
        print("*"*20)

In [18]:
%%time
semantic_search('Great location' , sentence_embeddings=sentence_embeddings , top_k=5)

Query: Great location
Top 5 most similar documents:
********************
Score: 0.9557

 Great rooms stunning architecture  ,  Free wifi only for guests booking direct with the hotel not via booking sites No guest lounge so you have to sit in reception or at cafe tables with a minimum spend of 12 50 per head during the day   

********************
********************
Score: 0.9557

 This property is in an excellent location a small boutique hotel rooms were very clean will definitely stay here again  , No Negative  

********************
********************
Score: 0.9114

 The staff and my room view  , No Negative  

********************
********************
Score: 0.8671

 Nothing Hotel awful will not be returning ,  Everything Old dirty needs a re vamp Pool was dirty could not even see through water Dirty  

********************
********************
Score: 0.8642

 Location and staff ,  I ve stayed in quite a few of the more fancy hotels in this area bearing in mind how expensive t

In [44]:
sentence_embeddings.shape[1]


768

# faiss

In [21]:
import faiss

## IndexFlatL2

In [23]:
%%time
query = ['Great location']
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d) #euclidean method.  - intialize
numpy_embeddings = sentence_embeddings.detach().cpu().numpy()
index.add(numpy_embeddings)

CPU times: user 790 ms, sys: 1.63 s, total: 2.42 s
Wall time: 2.23 s


In [24]:
index.is_trained

True

In [25]:
%%time
k = 3
xq = model.encode(
    query
    )
D, I = index.search(xq, k)  # search
[f'{i}: {sentences[i]}' for i in I[0]]

CPU times: user 559 ms, sys: 823 µs, total: 560 ms
Wall time: 557 ms


['122153:  This property is in an excellent location a small boutique hotel rooms were very clean will definitely stay here again  , No Negative  ',
 '320705:  Great rooms stunning architecture  ,  Free wifi only for guests booking direct with the hotel not via booking sites No guest lounge so you have to sit in reception or at cafe tables with a minimum spend of 12 50 per head during the day   ',
 '28617:  The staff and my room view  , No Negative  ']

## Quantization

In [32]:
nlist = 100
d = sentence_embeddings.shape[1]
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)
numpy_embeddings = sentence_embeddings.detach().cpu().numpy()

In [33]:

index.is_trained

False

In [34]:

index.train(numpy_embeddings)


In [35]:

index.is_trained

True

In [36]:

index.add(numpy_embeddings)
index.ntotal

499288

In [48]:
%%time
query = ['Great location']
xq = model.encode(
    query
    )

D, I = index.search(xq, k=3)
result = [f'{i}: {sentences[i]}' for i in I[0]]
result

CPU times: user 21.9 ms, sys: 0 ns, total: 21.9 ms
Wall time: 20.9 ms


['122153:  This property is in an excellent location a small boutique hotel rooms were very clean will definitely stay here again  , No Negative  ',
 '320705:  Great rooms stunning architecture  ,  Free wifi only for guests booking direct with the hotel not via booking sites No guest lounge so you have to sit in reception or at cafe tables with a minimum spend of 12 50 per head during the day   ',
 '28617:  The staff and my room view  , No Negative  ']

## IVF index


In [66]:
d = sentence_embeddings.shape[1]
quantizer = faiss.IndexFlatL2(d)
nlist = 100  # Number of clusters
ivf_index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [67]:
ivf_index.is_trained

False

In [68]:
ivf_index.train(numpy_embeddings)

In [69]:
ivf_index.is_trained

True

In [70]:
ivf_index.add(numpy_embeddings)
ivf_index.ntotal

499288

In [78]:
%%time
# Querying the IVF index
D, I = ivf_index.search(xq, k=2)
result = [f'{i}: {sentences[i]}' for i in I[0]]
result

CPU times: user 4.78 ms, sys: 0 ns, total: 4.78 ms
Wall time: 4.12 ms


['122153:  This property is in an excellent location a small boutique hotel rooms were very clean will definitely stay here again  , No Negative  ',
 '320705:  Great rooms stunning architecture  ,  Free wifi only for guests booking direct with the hotel not via booking sites No guest lounge so you have to sit in reception or at cafe tables with a minimum spend of 12 50 per head during the day   ']

## Hierarchical Navigable Small World (HNSW) Index

In [79]:
hnsw_index = faiss.IndexHNSWFlat(d, 5)  # 5 is the number of neighbors
hnsw_index.add(numpy_embeddings)
hnsw_index.ntotal

499288

In [ ]:
query = ['Great location']
xq = model.encode(
    query
    )

In [84]:
%%time

# Querying the HNSW index
D, I = hnsw_index.search(xq, k=2)
result = [f'{sentences[i]}' for i in I[0]]
result

CPU times: user 4.13 ms, sys: 0 ns, total: 4.13 ms
Wall time: 955 µs


[' This property is in an excellent location a small boutique hotel rooms were very clean will definitely stay here again  , No Negative  ',
 ' Great rooms stunning architecture  ,  Free wifi only for guests booking direct with the hotel not via booking sites No guest lounge so you have to sit in reception or at cafe tables with a minimum spend of 12 50 per head during the day   ',
 ' The staff and my room view  , No Negative  ']

## Composite Indexes (IVFPQ)  


In [81]:
# Composite Indexes (IVFPQ)

quantizer = faiss.IndexFlatL2(d)
ivfpq_index = faiss.IndexIVFPQ(quantizer, d, nlist, 4, 8)  # 4 subquantizers, 8 bits per subvector
ivfpq_index.train(numpy_embeddings)
ivfpq_index.add(numpy_embeddings)
ivfpq_index.ntotal

499288

In [ ]:
query = ['Great location']
xq = model.encode(
    query
    )

In [85]:
%%time
# Querying the IVFPQ index
D, I = ivfpq_index.search(xq, k=2)
result = [f'{sentences[i]}' for i in I[0]]
result

CPU times: user 1.63 ms, sys: 9 µs, total: 1.64 ms
Wall time: 1.02 ms


[' Great rooms stunning architecture  ,  Free wifi only for guests booking direct with the hotel not via booking sites No guest lounge so you have to sit in reception or at cafe tables with a minimum spend of 12 50 per head during the day   ',
 ' This property is in an excellent location a small boutique hotel rooms were very clean will definitely stay here again  , No Negative  ']

## GPU-specific Indexes (using the GPU version of FAISS)


In [ ]:

flat_index = faiss.IndexFlatL2(d)

res = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, flat_index)
gpu_index.add(numpy_embeddings)
gpu_index.ntotal

In [ ]:
query = ['Great location']
xq = model.encode(
    query
    )

In [ ]:
%%time
# Querying the GPU index
D, I = gpu_index.search(xq, k=2)
result = [f'{sentences[i]}' for i in I[0]]
result